# Oracle MCP Email Agent Workshop

## Building a Document Search and Email Automation Agent

### Introduction

This notebook demonstrates how to build an agentic email assistant that can search uploaded documents, look up recipients, and automatically draft or send emails using LangChain, Oracle GenAI, and MCP tool integration.

**In this workshop, you'll learn:**
1. Environment and dependency setup for Oracle, LangChain, and MCP
2. How to register and use modular tools through MCP
3. How to load, chunk, and embed documents for RAG (Retrieval-Augmented Generation)
4. How a GenAI agent orchestrates tool use for complex queries
5. How to run everything seamlessly in Jupyter/VS Code

Let's get started!

# Section 1: Environment Setup/ Improrts and Database Connnection

We first set up our Python environment and import the necessary libraries, including LangChain components, OracleDB drivers, and the MCP (Modular Command Processor) platform.

- We use OracleDB Python drivers, **not** the deprecated `cx_Oracle`, to support vector search.
- All tools and configurations are version-controlled and reproducible in this notebook.

## Imports and Configuration

In [4]:
import os
import nest_asyncio
import subprocess
from dotenv import load_dotenv
from tools9 import DatabaseOperations


# Enable nested asyncio loops for Jupyter
nest_asyncio.apply()

# Load .env with Oracle etc.
load_dotenv()

# (Optional) Silence HuggingFace tokenizer warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Oracle Database Connection

Connecting to Oracle database using the database username and password which are stored as environment variables (in .env file on linux)

In [5]:
db_ops = DatabaseOperations()
connected = db_ops.connect()
print("✅ Connected!" if connected else "❌ Connection Failed")


✅ Connected!


## 2. Start the MCP Tool Server in the Background

Before your notebook can use the agent tools (such as document search, email sending, and recipient lookup), you must start the MCP Tool Server. 

The MCP Tool Server is responsible for:
- Registering all your custom tools (with `@mcp.tool()` decorators)
- Handling communication between your notebook’s agent and the available tools

**How it works:**
- Start the tool server as a separate background process.
- The server listens for requests—such as RAG searches, recipient lookups, or email drafts—from your agent running in the notebook.

This approach ensures modularity: you can update your tools or restart the server independently of your notebook session.

**In this notebook, we will:**
- Use Python’s `subprocess` module to launch `server.py` in the background.
- Display the server code here for transparency and reproducibility.

**Tip:** If you modify your tools in `server.py`, restart the server process to load the changes.

In [6]:
import time

# Start the MCP server in background (make sure server.py is in this folder)
server_process = subprocess.Popen(['python', 'server.py'])
print(f"MCP server.py started in background with PID {server_process.pid}")

# Give the server a couple seconds to spin up
time.sleep(2)

MCP server.py started in background with PID 88909


## 3. Review the MCP Tool Server Code (`server.py`)

To promote transparency and reproducibility, it’s important to inspect the exact code that defines and registers the agent tools used by your MCP server.

**What you'll see here:**
- All tool registrations (`@mcp.tool()`), such as document chunking, RAG search, recipient lookup, and email functions
- Any configuration, imports, or shared state the server uses

By including the content of `server.py` directly in this notebook, anyone can understand or audit the logic available to your agent, and you can easily compare versions or document changes during development.


In [7]:
from IPython.display import Markdown, display

with open("server.py", "r") as f:
    server_code = f.read()
display(Markdown(f"```python\n{server_code}\n```"))

```python
from mcp.server.fastmcp import FastMCP
from tools9 import DatabaseOperations, fetch_recipients, send_email_function, extract_email_data_from_response

from tools9 import fetch_recipients, send_email_function, chunks_to_docs_wrapper
from typing import List
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores.oraclevs import OracleVS
import os
mcp = FastMCP("EmailAssistant")

global_vector_store = None

def set_vector_store(store: OracleVS):
    global global_vector_store
    global_vector_store = store


def get_vector_store():
    global global_vector_store   # ← Add this line
    if global_vector_store:
        return global_vector_store

    
    try:
        db_ops = DatabaseOperations()
        if not db_ops.connect():
            raise Exception("DB connect failed")

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        global_vector_store = OracleVS(
        embedding_function=embeddings,
        client=db_ops.connection,
        table_name="MY_DEMO4",
        distance_strategy=DistanceStrategy.COSINE,
        )
        return global_vector_store
    except Exception as e:
        print("VectorStore init failed:", e)
        return None



@mcp.tool()
def lookup_recipients(name: str):
    return fetch_recipients(name)

# @mcp.tool()
# def prepare_and_send_email(to: str, subject: str, message: str):
#     return send_email_function({"to": to, "subject": subject, "message": message})


@mcp.tool()
def oracle_connect() -> str:
    """
    Checks and returns Oracle DB connection status.
    """
    try:
        db_ops = DatabaseOperations()
        if db_ops.connect():
            print("Oracle connection successful!")
            return db_ops.connection
        return None
    except Exception as e:
        print(f"Oracle connection failed: {str(e)}")
        return None    

@mcp.tool()
def extract_email_fields_from_response(response_text: str) -> dict:
    """
    Extracts email fields (to, subject, message) from an AI-generated response.

    Input:
    - response_text: A string containing the AI assistant's output.

    Output:
    - A dictionary with keys: "to", "subject", "message"
    """
    try:
        return extract_email_data_from_response(response_text)
    except Exception as e:
        return {"error": f"Failed to extract email data: {str(e)}"}


@mcp.tool()
def store_text_chunks(file_path: str) -> str:
    """Split text and store as embeddings in Oracle Vector Store"""
    try:
        db_ops = DatabaseOperations()
        
        if not db_ops.connect():
            return "❌ Oracle connection failed."

        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            raw_text = f.read()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            chunks = text_splitter.split_text(raw_text)
            file_name = os.path.basename(file_path)
            docs = [
                chunks_to_docs_wrapper({'id': f"{file_name}_{i}", 'link': f"{file_name} - Chunk {i}", 'text': chunk})
                for i, chunk in enumerate(chunks)
            ]


            embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            vector_store = OracleVS.from_documents(
                docs, embeddings, client=db_ops.connection,
                table_name="MY_DEMO4", distance_strategy=DistanceStrategy.COSINE)
            
            # OracleVS(
            #     embedding_function=embeddings,
            #     client=db_ops.connection,
            #     table_name="MY_DEMO4",
            #     distance_strategy=DistanceStrategy.COSINE,
            # )

            set_vector_store(vector_store)

            return f"✅ Stored {len(docs)} chunks from {file_name}"

    except Exception as e:
        return f"❌ Error: {str(e)}"

@mcp.tool()
def rag_search(query: str) -> str:
    """
    Retrieve relevant information from user-uploaded documents stored in the Oracle Vector Store.

    Use this tool whenever a user asks a question that may be answered from the uploaded documents
    (e.g., HR policy files, contracts, technical manuals, PDF uploads, etc.).

    The tool performs a semantic similarity search over the embedded document chunks and returns
    the top 5 most relevant text snippets.

    Input:
    - A natural language question or topic from the user.

    Output:
    - A formatted string combining the most relevant document excerpts.

    Examples:
    - "What is the leave policy for new employees?"
    - "Summarize the refund terms in the uploaded contract"
    - "Find safety precautions mentioned in the manual"
    """
    try:
        # Load vector store (or access from persistent source if needed)
        vector_store = get_vector_store()
        if vector_store is None:
            return "❌ No documents have been indexed yet."

        docs = vector_store.similarity_search(query, k=5)
        return "\n".join([doc.page_content for doc in docs])
    except Exception as e:
        return f"❌ Error during document search: {str(e)}"


if __name__ == "__main__":

    print("Starting MCP Agentic Server...")
    mcp.run(transport="stdio")


```

## 4. MCP Agent & GenAI Setup

Now that the MCP Tool Server is running and our tools are registered, we'll set up the intelligent agent that will use these tools to solve user queries.

**What happens in this step:**
- Connect securely to the running MCP Tool Server from within the notebook.
- Dynamically retrieve the latest version of all registered tools (such as `rag_search`, recipient lookup, and email functions).
- Initialize a powerful GenAI (Generative AI) large language model (LLM) to act as the agent's reasoning engine.
- Combine the tools and LLM into an agentic workflow: the GenAI agent will understand user prompts, plan which tools to use, and execute complex tasks automatically.

This architecture allows the agent to orchestrate multiple tools seamlessly—retrieving documents, summarizing content, finding recipients, and drafting or sending emails—all based on natural language prompts.

In [8]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain.schema import HumanMessage

# Set up the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# This param is required by the package, but since our server is already running, we'll set connect=False below
server_params = StdioServerParameters(command="python", args=["server.py"])

async def run_mcp_agent(prompt):
    # Connect to the already-launched MCP tool server (stdin/stdout)
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            agent = create_react_agent(llm, tools)
            response = await agent.ainvoke({
                "messages": [HumanMessage(prompt)]
            })
            return response["messages"][-1].content

## 5. Index a Test Document

To enable document search and retrieval-augmented generation (RAG), the agent requires access to relevant documents that have been chunked and embedded in the Oracle Vector Store.

**In this step:**
- We'll create (or upload) a sample policy document.
- The agent will use the corresponding MCP tool (for example, `store_text_chunks`) to split the document into manageable text chunks and store their semantic embeddings in the Oracle DB vector store.
- These embeddings will enable fast, accurate semantic search when querying for information later.

Storing documents in this way allows our agent to understand and answer questions based on actual company policies, contracts, manuals, or any other proprietary documents you choose to index.


In [9]:
# Create a simple policy file
test_file_path = "sample_policy.txt"
policy_text = """
EMPLOYEE LEAVE POLICY (v2023)

SECTION 1: ANNUAL LEAVE ENTITLEMENTS

1.1 All full-time employees are entitled to 15 working days of paid annual leave per calendar year.
1.2 Leave accrues at a rate of 1.25 days per month of service.
1.3 Maximum carryover of unused leave is 5 days into the next calendar year.

SECTION 2: SICK LEAVE

2.1 Employees receive 10 days of paid sick leave annually.
2.2 Medical certificate required for absences exceeding 3 consecutive days.
2.3 Unused sick leave does not carry over to the next year.

SECTION 3: PARENTAL LEAVE

3.1 Maternity leave: 12 weeks paid leave for new mothers.
3.2 Paternity leave: 4 weeks paid leave for new fathers.
3.3 Adoption leave: 8 weeks paid leave for adoptive parents.

SECTION 4: SPECIAL CIRCUMSTANCES

4.1 Bereavement leave: 5 days for immediate family members.
4.2 Jury duty: Full pay for duration of service.
4.3 Military leave: Protected unpaid leave for reservists.

CONTACT INFORMATION:

- HR Department: hr@company.com
- Leave Requests: leave@company.com
- Emergency Contact: +1 (555) 123-4567

POLICY UPDATES:

This policy is reviewed annually. Last updated: January 2023.
For interpretation questions, contact the HR Director.
"""

with open(test_file_path, "w") as f:
    f.write(policy_text)

print(f"Created sample policy document with {len(policy_text.split())} words at: {test_file_path}")

with open(test_file_path, "w") as f:
    f.write("Employees are entitled to 15 days annual leave. For more details, contact HR at hr@company.com.")

# Ask the MCP agent to index the file using the document chunking tool
result = await run_mcp_agent(f"Please index this document: {test_file_path}")
print(result)

Created sample policy document with 187 words at: sample_policy.txt
Please provide the document "sample_policy.txt" so that I can proceed with indexing it.


## 6. Run an End-to-End Agent Example

In this step, you’ll see the full power of the GenAI agent in action:  
By providing a single natural language prompt, the agent will:

1. Use the MCP tools to query the indexed documents (via semantic search/RAG),
2. Summarize the relevant information,
3. Lookup the recipient’s details as needed,
4. And automatically draft and (optionally) send an email with the findings.

This demonstrates true agentic orchestration:  
The system transparently selects and invokes multiple tools, combining their outputs to fulfill a complex, real-world workflow—all from a single user request.


In [10]:
user_query = (
    "Find the leave policy from the indexed documents and email HR with the summary."
)
result = await run_mcp_agent(user_query)
print("AGENT RESPONSE:\n", result)

AGENT RESPONSE:
 I have extracted the email fields for you:

- **To:** hr@company.com
- **Subject:** Information
- **Message:** Please email HR with the following summary of the leave policy: Employees are entitled to 15 days annual leave. For more details, contact HR at hr@company.com.

You can now send this email to HR.


## 7. Try RAG, Recipient Lookup, & Email Drafts

Now let's interactively test the core MCP tools that power our GenAI agent:

- **RAG Search:** Extract answers to company-specific questions from your indexed policy documents using Retrieval-Augmented Generation (RAG).
- **Recipient Lookup:** Quickly find the email address or contact info for team members or departments (e.g., "HR").
- **Email Drafting:** Automatically generate email drafts that summarize or forward policies and insights to the right recipients.

In this section, you'll run prompts that directly demonstrate each tool's capability.  
You'll see how the agent seamlessly orchestrates document search and business communication—all enabled by your modular MCP tool framework.

In [11]:
print("---\nRAG Search Example:")
result = await run_mcp_agent("What is the leave policy?")
print(result)

print("---\nRecipient Lookup Example:")
result = await run_mcp_agent("Find the email for Ashu.")
print(result)

print("---\nEmail Draft Example:")
result = await run_mcp_agent("Send a draft email about annual leave policy to Ashu.")
print(result)

---
RAG Search Example:
The leave policy entitles employees to 15 days of annual leave. For more detailed information, you can contact HR at hr@company.com.
---
Recipient Lookup Example:
The email for Ashu is ashu.kumar@oracle.com.
---
Email Draft Example:
Here's a draft email about the annual leave policy for Ashu:

---

**To:** ashu.kumar@oracle.com

**Subject:** Update on Annual Leave Policy

Hi Ashu,

I hope this message finds you well. I wanted to inform you about the recent updates to our annual leave policy. Please find the details below:

1. All employees are entitled to 20 days of paid annual leave per year.
2. Leave must be scheduled in advance and approved by your manager.
3. Unused leave days can be carried over to the next year, up to a maximum of 10 days.
4. For any leave exceeding 5 consecutive days, a formal request must be submitted.

Please let me know if you have any questions or need further clarification.

Best regards,

[Your Name]


## 8. Show Oracle/GenAI Features via RAG

A powerful advantage of this agentic architecture is the ability to surface product or policy features directly from your indexed documents using semantic search and GenAI summarization.

**In this step:**
- You’ll ask the agent to list key features of Oracle 23ai (or any product, policy, or technical topic you’ve indexed).
- The agent will use the RAG (Retrieval-Augmented Generation) tool to retrieve relevant document chunks and synthesize a summary, ensuring the response is both accurate and grounded in your actual content.

This capability is ideal for:
- Quickly briefing users on new software features
- Surfacing business policy changes
- Enabling dynamic, context-aware onboarding and knowledge management

In [12]:
prompt = "List 5 new features of Oracle 23ai from the indexed documents."
result = await run_mcp_agent(prompt)
print(result)

It seems that the document search returned information unrelated to Oracle 23ai features. Please ensure that the relevant documents are uploaded and indexed, or provide more specific details about the features you are interested in.


## 9. Shut Down MCP Server

When you’re finished using the agent and running notebook experiments, it’s important to stop the background MCP Tool Server process you started earlier.

In [13]:
server_process.terminate()
print("Stopped MCP server process.")

Stopped MCP server process.
